# 인스타그램 이미지 크롤링

참고 : https://yeowool0217.tistory.com/564

In [1]:
import pandas as pd
import time 
import re 
import time
import urllib
from bs4 import BeautifulSoup 
from urllib.request import urlopen, Request
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from urllib.error import HTTPError

In [2]:
search = input('검색어를 입력하세요:')
url = 'https://www.instagram.com/explore/tags/'+ str(urllib.parse.quote(search)) +'/'

driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(3)

driver.find_element_by_css_selector('div.v1Nh3.kIKUG._bz0w').click()

검색어를 입력하세요:오오티디


In [4]:
SCROLL_PAUSE_TIME = 1.0
reallink = []

while True:
    pageString = driver.page_source
    bsObj = BeautifulSoup(pageString, "lxml")

    for link1 in bsObj.find_all(name="div",attrs={"class":"Nnq7C weEfm"}):
        title = link1.select('a')[0] 
        real = title.attrs['href']
        reallink.append(real) 
        
        title = link1.select('a')[1] 
        real = title.attrs['href']
        reallink.append(real) 
        
        title = link1.select('a')[2] 
        real = title.attrs['href']
        reallink.append(real) 

    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

In [5]:
reallink = list(set(reallink))
print(reallink)
print("총", len(reallink), "개의 URL 수집")

['/p/CBajtbzAaVI/', '/p/CCiobcvjj77/', '/p/CCif7kjDf2M/', '/p/CCifhJ5Dh0I/', '/p/CCiocNXDtTg/', '/p/CCiobZ6l9HD/', '/p/CCioV56hmND/', '/p/CCiiWnDpmNi/', '/p/CCiL6UZFibw/', '/p/CCijY_ep6su/', '/p/CCiheA0h7I7/', '/p/CCioG3SJNaI/', '/p/CCiYr3npmhm/', '/p/CCinjWAJz9C/', '/p/CCioUZul_Mb/', '/p/CCimrYvgqmQ/', '/p/CBNnHX2AcQl/', '/p/CCigoJQHQf9/', '/p/CCioVd0JmR1/', '/p/CCioex1FTt-/', '/p/CCil67qj5G1/', '/p/CCiTPP4hlCj/', '/p/CCgxGnKlegd/', '/p/CCijDO_FE9F/', '/p/CCioXfwJ-MV/', '/p/CCilcYrD_93/', '/p/CCiocNvjBRI/', '/p/CCiNKQ5jDSD/', '/p/CCia4K3p1SN/', '/p/CCP3AUrBJip/', '/p/CCh739aFK06/', '/p/CChvLiPleOz/', '/p/CCiHjo3HiVx/', '/p/CCKQcKzJPSp/', '/p/CCioSb_Jlbc/', '/p/CBbw2A8AzM0/', '/p/CCiod9FJWfc/', '/p/CCioKYzj_si/', '/p/B-quWMfJB9O/', '/p/CCioSKvBsjk/', '/p/B-4MhidjUX2/', '/p/CCiRvv5Fs5l/', '/p/CCioSlEJsSE/', '/p/CBuFNPVAKHR/', '/p/CCioeknHJjh/', '/p/CCihidqFk9L/', '/p/CCioTefpSI_/', '/p/CCTBVPwHEH_/', '/p/CCimPRsDBVb/', '/p/CCifP98loq7/', '/p/CCioLZfAfAg/', '/p/CCioWpbnCXn/', '/p/CCiWO8s

In [ ]:
csvtext = []

for i in range(0, len(reallink)):
    if i % 10 == 0:
        print(i)
    csvtext.append([])
    req = Request('https://www.instagram.com/p' + reallink[i], headers={'User-Agent':'Mozilla/5.0'})
    
    try:
        webpage = urlopen(req).read()
    except HTTPError:
        print(req, 'HTTP Error 404: Not Found')
        reallink.remove(reallink[i])
        
    soup = BeautifulSoup(webpage, 'lxml', from_encoding='utf-8')
    soup1 = soup.find('meta', attrs={'property':'og:description'})
    
    reallink1 = soup1['content']
    reallink1 = reallink1[reallink1.find('@') + 1 : reallink1.find(')')]
    reallink1 = reallink1[:10]
    if reallink1 == '':
        reallink1 = 'Null'
    csvtext[i].append(reallink1)
    
    for reallink2 in soup.find_all('meta', attrs={'property':'instapp:hashtags'}):
        reallink2 = reallink2['content']
        csvtext[i].append(reallink2)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
<urllib.request.Request object at 0x00000200A323AA88> HTTP Error 404: Not Found
190
200
210
220
230
240
250
260


In [ ]:
data = pd.DataFrame(csvtext)
data.to_csv(str(search) +'.txt', encoding='utf-8')

In [ ]:
data = pd.read_csv(str(search) +'.txt')
data.head()

In [ ]:
img = []
date = []

for i, v in enumerate(reallink):
    if i % 10 == 0:
        print(i)
    url = 'https://www.instagram.com/'+str(v)
    driver.get(url)

    while(1):
        time.sleep(1)
        pageString = driver.page_source 
        soup = BeautifulSoup(pageString, "lxml")
        imgs = soup.select('img')[1]
        imgs = imgs.attrs['src']
        img.append(imgs)
        
        try :
            temp = driver.find_element_by_css_selector("time._1o9PC.Nzb55" ).text
            date.append(temp)
            print(temp)
            driver.find_element_by_class_name("coreSpriteRightChevron").click()

        except NoSuchElementException:
            break

driver.close()
print('이미지 주소 수집완료')
img = list(set(img))

In [ ]:
n = 0
for i, v in enumerate(img):
    filename = str(n)+search+date[i]+'.png'
    urllib.request.urlretrieve(v, filename)
    n += 1
print('저장성공')